In [ ]:
# General setup, as explained earlier
import os
from pprint import pprint
from urllib3.util.retry import Retry

import requests
from requests.adapters import HTTPAdapter

PLANET_API_URL = 'https://api.planet.com/data/v1'

def setup_session(api_key=None):
    """
    Initialize a requests.Session that handles Planet api key auth and retries.
    
    :param str api_key:
        A Planet api key. Will be read from the PL_API_KEY env var if not specified.
    
    :returns requests.Session session:
        A Session instance optimized for use with Planet's api.
    """
    if api_key is None:
        api_key = os.getenv('PL_API_KEY')

    session = requests.Session()
    session.auth = (api_key, '')

    retries = Retry(total=5,
                    backoff_factor=0.2,  
                    status_forcelist=[429])

    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

session = setup_session() # Or pass in an api key if the environment variable isn't set

Item-Types Endpoint and Planet Data Structure
---------------------------------------------------------------------

In our last exercise we used a somewhat random-looking url to demonstrate authentication principles:
https://api.planet.com/data/v1/item-types/PSScene4Band/items/20191010_183406_0f28

That url uses the `item-types/` endpoint of the data API to request information about a scene. Let's take a closer look at what the url actually returns:

In [ ]:
url = 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20191010_183406_0f28'

# Going to cheat a bit and just assume the request will work...
info = session.get(url).json()

pprint(info)

This is all of the metadata for this scene in our API.  The docs for this API endpoint are: https://developers.planet.com/docs/api/items-assets/

Let's start out with the fun stuff -- We'll grab the thumbnail for the scene and display it:

In [ ]:
from IPython.display import Image
Image(session.get(info['_links']['thumbnail']).content)

Different Item Types
------------------------------

This is a good place for a bit of a refresher on a couple of key concepts for Planet data.  We have multiple different item types.  

You can see which item-types you have access to by requesting the root `item-types` endpoint:

In [ ]:
pprint(session.get('https://api.planet.com/data/v1/item-types/').json())

Most of these represent data from different satellites, but for many of our satellites, the same imagery may be exposed as different itemtypes.  For example, for Planetscope, the same underlying data can be represented as `PSScene4Band`, `PSScene3Band`, and `PSOrthoTile` item types.

The key for this is really geometry. If a representation of the data has a different outline, it must be repesented as a different item type even when it is from the same satellite.  To demonstrate this more concretely, let's compare the `PSScene3Band` and `PSScene4Band` geometries for the same scene:

In [ ]:
def get_info(itemtype, sceneid):
    url = '{}/item-types/{}/items/{}'.format(PLANET_API_URL, itemtype, sceneid)
    response = session.get(url)
    response.raise_for_status()
    return response.json()
    

scene = '20191010_183406_0f28'
info_4band = get_info('PSScene4Band', scene)
info_3band = get_info('PSScene3Band', scene)

print('PSScene4Band Geometry')
pprint(info_4band['geometry'])

print('\nPSScene3Band Geometry')
pprint(info_3band['geometry'])

Notice that the footprints for the same sceneid are slightly different between the two different item types.  The `PSScene4Band` version has a more complex outline (small "stair steps") and is a tiny bit smaller. The differences in area are quite small in this case, but it's not uncommon to see 4-band imagery for a scene be much smaller than 3-band imagery.  This is because our satellites don't actually collect 4-band imagery directly.  The NIR band comes from a different scene taken a second or so later.  Only the pixels that have overlapping NIR data from other scenes can be used. 

Let's display the footprints to compare:

In [ ]:
import json
import geojsonio

data = {'type': 'FeatureCollection',
        'features': [info_3band, info_4band]
       }
geojsonio.display(json.dumps(data))

Okay, yeah, the differences are really subtle there, but let's look at a more extreme example:

In [ ]:
# Coastal scenes tend to have larger differences
reef_scene = '20191004_192044_0f33'
data = {'type': 'FeatureCollection',
        'features': [get_info('PSScene4Band', reef_scene),
                     get_info('PSScene3Band', reef_scene)]
       }
geojsonio.display(json.dumps(data))

Scene Metadata
------------------------
You may recall that our earlier request for a scene's metadata returned quite a bit of information under the `properties` key:

In [ ]:
info = get_info('PSScene4Band', scene)

pprint(info['properties'])

Let's take a quick look at what these metadata fields in `properties` are.  Note that each item type has a different set of metadata.  There are detail docs for each item type at https://developers.planet.com, e.g.:

https://developers.planet.com/docs/api/psscene3band/

|Property Name|Description|Type|
|:--- |:--- |:--- |
|id|Globally unique item identifier.|string|
|_permissions|The permissions that the authenticated user has to the item.|array|
|item_type|Name of the item type.|string|
|provider|Name of the item provider (e.g. “planetscope”, ”rapideye”).|string|
|satellite_id|Globally unique satellite identifier.|string|
|instrument|Name of the satellite instrument used to collect the image.|string|
|strip_id|The unique identifier of the image stripe that the item came from.|string|
|geometry|Geographic boundary, formatted as a GeoJSON polygon.|json|
|cloud_cover|Average percentage of cloud coverage.|double|
|gsd|Ground sample distance - the distance between pixel centers as measured on the ground in meters.|double|
|pixel_resolution|Pixel resolution of the imagery in meters.|double|
|epsg_code|Ortho tile grid cell that the item is located in (not used if Scene).|integer|
|sun_azimuth|The angle of the sun, as seen by the observer, measured clockwise from the north (0 - 360).|double|
|sun_elevation|The angle of the sun above the horizon (0 - 90).|double|
|view_angle|The satellite's across-track, off-nadir viewing angle. Positive numbers denote east, negative numbers denote west (-25 - +25).|double|
|rows|Number of rows in the image.|integer|
|columns|Number of columns in the image.|integer|
|origin_x|ULX coordinate of the extent of the data. The coordinate references the top left corner of the top left pixel.|double|
|origin_y|ULY coordinate of the extent of the data. The coordinate references the top left corner of the top left pixel.|double|
|quality_category|Planet image quality metric: Standard or Test. To qualify for "standard" image quality an image must meet a variety of quality standards, for example: PAN motion blur less than 1.15 pixels, compression bits per pixel less than 3. If the image does not meet these criteria it is considered "test" quality.|string|
|anomalous_pixels|Percentage of pixels that may have errors. Represented spatially in the UDM.|double|
|usable_data|Percentage of pixels that are usable, subtracting cloud cover and black fill.|double|
|ground_control|Positional accuracy of the item. If the item has uncertain positional accuracy, this value will be false.|boolean|
|acquired|Timestamp that the item was captured.|datetime|
|published|Timestamp that the item was published to the Planet API.|datetime|
|updated|Timestamp that the item record was last updated.|datetime|

To demonstrate that these vary by ItemType, compare the table above to the metadata for a `SkySatScene`:

https://developers.planet.com/docs/api/skysatscene/

|Property Name|Description|Type|
|:--- |:--- |:--- |
|id|Globally unique item identifier.|string|
|_permissions|The permissions that the authenticated user has to the item.|array|
|item_type|Name of the item type.|string|
|provider|Name of the item provider (e.g. “planetscope”, ”rapideye”).|string|
|satellite_id|Globally unique satellite identifier.|string|
|strip_id|The unique identifier of the image stripe that the item came from.|string|
|geometry|Geographic boundary, formatted as a GeoJSON polygon.|json|
|cloud_cover|Average percentage of cloud coverage.|double|
|gsd|Ground sample distance - the distance between pixel centers as measured on the ground in meters.|double|
|sun_azimuth|The angle of the sun, as seen by the observer, measured clockwise from the north (0 - 360).|double|
|sun_elevation|The angle of the sun above the horizon (0 - 90).|double|
|view_angle|The satellite's across-track, off-nadir viewing angle. Positive numbers denote east, negative numbers denote west (-25 - +25).|double|
|ground_control|Positional accuracy of the item. If the item has uncertain positional accuracy, this value will be false.|boolean|
|acquired|Timestamp that the item was captured.|datetime|
|published|Timestamp that the item was published to the Planet API.|datetime|
|updated|Timestamp that the item record was last updated.|datetime|


In the next section, we'll be filtering and searching based on these metadata fields.  You'll often want to keep the metadata docs for the item type you're working with nearby when using our data API. 

---

Assets
----------

Assets represent specific files.  These might be different processing levels of the same underlying imagery, or things like XML metadata and/or usable data masks.

In "Planet terms" you need an item type (e.g. `PSScene4Band`) a scene ID (e.g. `20191010_183406_0f28`) _and_ an asset type (e.g. `analytic_sr`) to uniquely identify the data you're working with.  

In other words, the asset is the data you'll actually download.  

It's important to note that not all scenes of the same item type will have the same assets.  Some assets are only available under certain conditions.  For example, scenes from satellites that haven't been fully calibrated won't have `analytic` or `analytic_sr` assets, and scenes before mid-2018 generally won't have `udm2` (a.k.a. "cloud 2.0") assets.  

Let's explore how to check which assets are available for a specific scene (we'll come back to this later):

In [ ]:
# Just repeating what we did earlier as a reminder

def get_info(itemtype, sceneid):
    url = '{}/item-types/{}/items/{}'.format(PLANET_API_URL, itemtype, sceneid)
    response = session.get(url)
    response.raise_for_status()
    return response.json()
    

scene = '20191010_183406_0f28'
info_4band = get_info('PSScene4Band', scene)

# Now let's look at the _links section
pprint(info_4band['_links'])

In [ ]:
# And we can make a GET request to the `assets` item:
res = session.get(info_4band['_links']['assets'])
res.raise_for_status()

# The keys in the response are the assets that are available:
for key in res.json():
    print(key)

Each item type has different assets -- e.g. the `analytic_sr` asset currently availabe for `PSScene3Band` items. A detailed description of what these different assets are is available at https://developers.planet.com/docs/api/psscene4band/ 

|Asset Name|Description|
|:--- |:--- |
|analytic|Radiometrically calibrated GeoTiff suitable for analytic applications.|
|analytic_sr|Atmospherically corrected surface reflectance product.|
|analytic_xml|Analytic asset XML metadata file.|
|analytic_dn|Non-radiometrically calibrated GeoTiff suitable for analytic applications.|
|analytic_dn_xml|Analytic DN asset XML metadata file.|
|udm|Unusable Data Mask - Unusable data bit mask in GeoTIFF format for the visual and analytic scene assets.|
|basic_analytic|Top of atmosphere radiance (at sensor) and sensor corrected GeoTiff. Scene based framing and not projected to a cartographic projection.|
|basic_analytic_xml|Basic analytic asset XML metadata file.|
|basic_analytic_rpc|Rational Polynomial Coefficients text file used to orthorectify the basic_analytic asset.|
|basic_analytic_dn|Basic sensor corrected scene GeoTiff. Scene based framing and not projected to a cartographic projection.|
|basic_analytic_dn_xml|Basic analytic DN asset XML metadata file.|
|basic_analytic_dn_rpc|Rational Polynomial Coefficients text file used to orthorectify basic_analytic_dn asset.|
|basic_analytic_nitf|Top of atmosphere radiance (at sensor) and sensor corrected in NITF format. Scene based framing and not projected to a cartographic projection.|
|basic_analytic_xml_nitf|Basic analytic XML metadata file.|
|basic_analytic_rpc_nitf|Rational Polynomial Coefficients text file used to orthorectify the basic_analytic asset.|
|basic_analytic_dn_nitf|Basic sensor corrected scene NITF. Scene based framing and not projected to a cartographic projection.|
|basic_analytic_dn_xml_nitf|Basic analytic DN XML metadata file.|
|basic_analytic_dn_rpc_nitf|Rational Polynomial Coefficients text file used to orthorectify basic_analytic_dn_nitf asset.|
|basic_udm|Usable Data Mask - Usable data bit mask in GeoTIFF format for the basic analytic scene assets.|
|basic_udm2|Usable Data Mask 2.0. Read more about this new asset here.|